In [1]:
import sys
import numpy as np
import sklearn.linear_model as skl
import pickle

In [2]:
sub = 2
assert sub in [1,2,5,7]

In [3]:
import os
os.chdir(r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush')
print(os.getcwd())  # Verify the new working directory

C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush


In [4]:
train_path = 'Data/processed_nsddata/subj{:02d}/nsd_train_fmriavg_nsdgeneral_sub{}.npy'.format(sub,sub)
train_fmri = np.load(train_path)
test_path = 'Data/processed_nsddata/subj{:02d}/nsd_test_fmriavg_nsdgeneral_sub{}.npy'.format(sub,sub)
test_fmri = np.load(test_path)

In [5]:
## Preprocessing fMRI

train_fmri = train_fmri/300
test_fmri = test_fmri/300


norm_mean_train = np.mean(train_fmri, axis=0)
norm_scale_train = np.std(train_fmri, axis=0, ddof=1)
train_fmri = (train_fmri - norm_mean_train) / norm_scale_train
test_fmri = (test_fmri - norm_mean_train) / norm_scale_train

print(np.mean(train_fmri),np.std(train_fmri))
print(np.mean(test_fmri),np.std(test_fmri))

print(np.max(train_fmri),np.min(train_fmri))
print(np.max(test_fmri),np.min(test_fmri))

num_voxels, num_train, num_test = train_fmri.shape[1], len(train_fmri), len(test_fmri)


train_clip = np.load('Data/extracted_features/subj{:02d}/nsd_clipvision_train.npy'.format(sub))
test_clip = np.load('Data/extracted_features/subj{:02d}/nsd_clipvision_test.npy'.format(sub))

#train_clip = train_clip[:,1:,:]
num_samples,num_embed,num_dim = train_clip.shape

print("Training Regression")
reg_w = np.zeros((num_embed,num_dim,num_voxels)).astype(np.float32)
reg_b = np.zeros((num_embed,num_dim)).astype(np.float32)
pred_clip = np.zeros_like(test_clip)
for i in range(num_embed):


    reg = skl.Ridge(alpha=60000, max_iter=50000, fit_intercept=True)
    reg.fit(train_fmri, train_clip[:,i])
    reg_w[i] = reg.coef_
    reg_b[i] = reg.intercept_

    pred_test_latent = reg.predict(test_fmri)
    std_norm_test_latent = (pred_test_latent - np.mean(pred_test_latent,axis=0)) / np.std(pred_test_latent,axis=0)
    pred_clip[:,i] = std_norm_test_latent * np.std(train_clip[:,i],axis=0) + np.mean(train_clip[:,i],axis=0)

    print(i,reg.score(test_fmri,test_clip[:,i]))


np.save('Data/predicted_features/subj{:02d}/nsd_clipvision_predtest_nsdgeneral.npy'.format(sub),pred_clip)

datadict = {
    'weight' : reg_w,
    'bias' : reg_b,

}

with open('Data/regression_weights/subj{:02d}/clipvision_regression_weights.pkl'.format(sub),"wb") as f:
  pickle.dump(datadict,f)

-2.6905091170276335e-16 0.9999435586284333
-0.0230087925174602 0.9934162733048788
14.987235205038967 -14.95580829898399
13.847471245936827 -12.376000889458274
Training Regression
0 0.09784930483814197
1 0.05777666852896352
2 0.07014501402653438
3 0.07566023770060405
4 0.09032659758289836
5 0.08925128330178644
6 0.07701602706185608
7 0.09044332527316033
8 0.0733403692630193
9 0.02716740971876865
10 0.08903137132661376
11 0.06714660461036172
12 0.059237146495996845
13 0.07061084897940555
14 0.08285483461928483
15 0.0629736212173565
16 0.06655152589772198
17 0.07846352599227059
18 0.0471653001662338
19 0.05566232519644807
20 0.05993005428534639
21 0.060188377402281105
22 0.08636598960147791
23 0.040849927220323975
24 0.05208291506253188
25 0.06751347126820784
26 0.03187985992482465
27 0.037551817021612
28 0.02856772320531656
29 0.06817080830138966
30 0.04016248458334998
31 0.090827616482078
32 0.07190163605330734
33 0.08465106712751058
34 0.06236066817682632
35 0.06536778362435704
36 0.05

In [6]:
import os
import numpy as np
import pickle

base_path = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results'

# === Save predicted CLIP-vision features ===
pred_path = os.path.join(base_path, 'predicted_features', f'subj{sub:02d}')
os.makedirs(pred_path, exist_ok=True)

pred_file = os.path.join(pred_path, 'nsd_clipvision_predtest_nsdgeneral.npy')
np.save(pred_file, pred_clip)

# === Save regression weights for CLIP-vision ===
reg_path = os.path.join(base_path, 'regression_weights', f'subj{sub:02d}')
os.makedirs(reg_path, exist_ok=True)

datadict = {
    'weight': reg_w,
    'bias': reg_b,
}

reg_file = os.path.join(reg_path, 'clipvision_regression_weights.pkl')
with open(reg_file, "wb") as f:
    pickle.dump(datadict, f)
